# Costa Rica Institute of Technology
* Course: MP-6122 Pattern Recognition
* Student: Jose Martinez Hdez
* Course: Data Science 
* Year: 2022
* Laboratory 2: Supervised Learning Models

# Dataset

# Preprocessing

# Benchmarking Analysis

## Model 1: Linear Regression

## Model 2: Linear Regression

## Model 3: Linear Regression

## Model 4: Linear Regression

## Model 5: Linear Regression

## Model 6: Linear Regression

# Metrics

# Model's Comparison